In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from shutil import copyfile

In [16]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/data/ortho/testset/' + PatientFile, header=None)
#     df_cleaned = pd.read_csv(PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

def VisualizeItemCleaned(item, x, y, z):
    global FileWithItemsToDelete
    global FileWithItemsToAdjust
    global FileWithItemsToSplitAgain
    
    
    
    FileName = item['FileName'].split('/')[5]
    Split = item['Split']
    JunkBefore = item['RemoveJunkBefore']
    JunkAfter = item['RemoveJunkAfter']
    User = item['User']
    
    
    

    try:
        splitted = FileName.split('.')[0].split('_')
        Cat = int(splitted[0].split('Cat')[1])
        pat = int(splitted[1].split('pat')[1])
        met = int(splitted[2].split('meting')[1])
        oef = int(splitted[3].split('oef')[1])
        titel = '%s %s %s %s by: %s' % (Cat, pat, met, oef, User)
    except(IndexError):
        oef = 'failed'
        titel = oef

    origdata = HeaderMaker(FileName)
    MaxFrame = len(origdata) - 1
    
    
    
    
    start = 0
    end = MaxFrame + 1
    
    if (JunkBefore > 0 and JunkBefore < MaxFrame):
#         plt.axvline(x=JunkBefore, color='b')
        start = JunkBefore
        
    if (JunkAfter > 0 and JunkAfter < MaxFrame):
#         plt.axvline(x=JunkAfter, color='g')
        end = JunkAfter
    data = origdata.iloc[start:end]
        
    if (Split > JunkBefore and Split < MaxFrame):
        data = [data.iloc[:Split-JunkBefore], data.iloc[Split-JunkBefore:]]
    else:
        data = [data]
    
#     print(titel)
    if ((Cat == 3) & (pat == 31) & (met == 1) & (oef == 5)):
        print('Our oefening')
    
    
    for num, plaatje in enumerate(data):
        splitnum = num + 1
        titel = FileName.split('.csv')[0] + '_split%s.csv' % (splitnum)
        
        delete = FileWithItemsToDelete.loc[(FileWithItemsToDelete['Cat'] == Cat) & (FileWithItemsToDelete['pat'] == pat) & (FileWithItemsToDelete['meting'] == met) & (FileWithItemsToDelete['oef'] == oef) & (FileWithItemsToDelete['Split'] == splitnum)]
        if delete.empty == False:
            print('Delete this one')
            continue
            
        adjust = FileWithItemsToAdjust.loc[(FileWithItemsToAdjust['Cat'] == Cat) & (FileWithItemsToAdjust['pat'] == pat) & (FileWithItemsToAdjust['meting'] == met) & (FileWithItemsToAdjust['oef'] == oef) & (FileWithItemsToAdjust['split'] == splitnum)]
        if adjust.empty == False:
            
            NewBegin = adjust['begin'].values[0]
            NewEnd = adjust['eind'].values[0]
            
            if NewBegin == 0:
                NewBegin = start
            if NewEnd == 0:
                NewEnd = end
            
            plaatje = origdata[NewBegin:NewEnd]
            print('Gecorigeerd')
        
        
        SplitAgain = FileWithItemsToSplitAgain.loc[(FileWithItemsToSplitAgain['Cat'] == Cat) & (FileWithItemsToSplitAgain['Pat'] == pat) & (FileWithItemsToSplitAgain['Meting'] == met) & (FileWithItemsToSplitAgain['Oef'] == oef) & (FileWithItemsToSplitAgain['Split'] == splitnum)]
        if SplitAgain.empty == False:
            NewBegin = SplitAgain['Begin1'].values[0]
            NewEnd2 = SplitAgain['Eind2'].values[0]
            
            if NewBegin == 0:
                NewBegin = start
            if NewEnd2 == 0:
                NewEnd2 = end
            
            Plaatje1 = origdata[NewBegin:SplitAgain['Eind1'].values[0]]
            Plaatje2 = origdata[SplitAgain['Begin2'].values[0]:NewEnd2]
            
            Plaatje1.to_csv('/datc/ortho/Cleaning/Step2 - testset/%s' % FileName.split('.csv')[0] + '_split%s.csv' % (splitnum+10), index=False, header=False)
            Plaatje2.to_csv('/datc/ortho/Cleaning/Step2 - testset/%s' % FileName.split('.csv')[0] + '_split%s.csv' % (splitnum+11), index=False, header=False)
            print('Splitted a splitted plaatje')
            continue

        
#         xlist = plaatje[x]
#         ylist = plaatje[y]
#         zlist = plaatje[z]

#         fig = plt.figure()
#         plt.title(titel)
#     #         plt.ylim((-180,180))
#         plt.plot(xlist, color = 'red')
#         plt.plot(ylist, color = 'blue')
#         plt.plot(zlist, color = 'green')
#         plt.legend()
        print('Normal')
        plaatje.to_csv('/datc/ortho/Cleaning/Step2 - testset/%s' % titel, index=False, header=False)

    

In [17]:
FileWithItemsToDelete =pd.read_csv('./Corrections/Split to delete.csv', sep=';')
FileWithItemsToAdjust =pd.read_csv('./Corrections/Split splitted.csv', sep=';')
FileWithItemsToSplitAgain =pd.read_csv('./Corrections/Split non splitted.csv', sep=';')

# MetaData = pd.read_csv('AllMetaData - All.csv')
MetaData = pd.read_csv('Metadata Testset.csv')

Files = MetaData['FileName'].apply(lambda x: x.split('/')[5])


# x, y, z = ("thorax_r_x","thorax_r_y", "thorax_r_z")
# x, y, z = ("clavicula_r_x", "clavicula_r_y", "clavicula_r_z")
# x, y, z = ("scapula_r_x", "scapula_r_y", "scapula_r_z")
x, y, z = ("humerus_r_x", "humerus_r_y", "humerus_r_z")
# VisualizeItemsCleaned(Files, x, y, z)
for index, row in MetaData.iterrows():

    VisualizeItemCleaned(row, x, y, z)
    

print('Done')

Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal
Normal

# Step 3
Sort all

In [18]:
def alpha(file):
#     print('alpha')
    copyfile(datc + file, enddir + 'alpha/' + file)
    return
    
def bravo(file):
#     print('bravo')
    copyfile(datc + file, enddir + 'bravo/' + file)
    return
    
def charlie(file):
#     print('charlie')
    copyfile(datc + file, enddir + 'charlie/' + file)
    return
    
def delta(file):
#     print('delta')
    copyfile(datc + file, enddir + 'delta/' + file)
    return
    
def echo(file):
#     print('echo')
    copyfile(datc + file, enddir + 'echo/' + file)
    return

enddir = '/datc/ortho/Cleaned oef/testset/'

datc = '/data/ortho/testset/'

for file in listdir(datc):
    if 'meting' not in file:
        continue
    location = file
    splitted = file.split('.')[0].split('_')
    Cat = int(splitted[0].split('Cat')[1])
    pat = int(splitted[1].split('pat')[1])
    met = int(splitted[2].split('meting')[1])
    oef = int(splitted[3].split('oef')[1])
    try:
        split = int(splitted[4].split('split')[1])
    except(IndexError):
        split = 0
#     print(location)
#     break

    
    if Cat == 1:
        if oef in [2, 3]:
            alpha(location)
        elif oef in [4, 5]:
            bravo(location)
        elif oef in [6,7]:
            charlie(location)
        elif oef in [8,9]:
            delta(location)
        elif oef in [10,11]:
            echo(location)
        else:
            print('error: ', Cat, pat, met, oef, split)
            
    if Cat == 1:
        if oef in [2, 3]:
            alpha(location)
        elif oef in [4, 5]:
            bravo(location)
        elif oef in [6,7]:
            charlie(location)
        elif oef in [8,9]:
            delta(location)
        elif oef in [10,11]:
            echo(location)
        else:
            print('error: ', Cat, pat, met, oef, split)
    
    if Cat == 2:
        if oef in [2, 3]:
            alpha(location)
        elif oef in [4, 5]:
            bravo(location)
        elif oef in [6,7]:
            charlie(location)
        elif oef in [8,9]:
            delta(location)
        elif oef in [10,11]:
            echo(location)
        else:
            print('error: ', Cat, pat, met, oef, split)
    
    if Cat == 3:
        if oef in [2]:
            alpha(location)
        elif oef in [3]:
            bravo(location)
        elif oef in [4]:
            charlie(location)
        elif oef in [5]:
            delta(location)
        elif oef in [6]:
            echo(location)
        else:
            print('error: ', Cat, pat, met, oef, split)
    
    if Cat == 4:
        if oef in [2, 3, 4, 5]:
            alpha(location)
        elif oef in [6, 7, 8, 9]:
            bravo(location)
        elif oef in [10, 11, 12, 13]:
            charlie(location)
        elif oef in [14, 15, 16, 17]:
            delta(location)
        elif oef in [18, 19, 20, 21]:
            echo(location)
        else:
            print('error: ', Cat, pat, met, oef, split)
#     print(Cat, pat, met, oef, split)


error:  4 13 2 22 0
error:  4 13 1 22 0
